In [15]:
import asyncio
from langchain.agents import Tool
from yt_music import NotAuthenticated as yt
from langgraph.graph import StateGraph, END, START
from typing import Annotated, TypedDict, List, Optional, Any
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from pydantic import BaseModel
from IPython.display import Image, display
import gradio as gr
from yt_music import Authenticated as yt_oath
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools import StructuredTool

In [2]:
llm = ChatOpenAI(api_key = 'ollama', base_url = "http://127.0.0.1:11434/v1", model = 'qwen3:8b' )

In [3]:
memory = MemorySaver()

In [4]:
yt_music = yt()
yt_oath_user = yt_oath()

In [5]:
search_tool = Tool(
    name="search_music",
    description="use this tool to search for a song on YouTube Music",
    func=yt_music.search_song  # async function
)


In [ ]:
# result = search_tool.invoke("The kings affirmation Iniko")
# print(result)

[{'song_id': 'bVwPpsEqmm8', 'title': "The King's Affirmation - Chill Mix (feat. Reuel Williams)", 'duration': '3:07', 'artists': ['Iniko'], 'album': ''}, {'song_id': 'S45rTCJwsWE', 'title': "The King's Affirmation", 'duration': None, 'artists': ['Iniko'], 'album': ''}, {'song_id': 'lJcO6gh-wzg', 'title': 'Jericho', 'duration': None, 'artists': ['Iniko'], 'album': ''}, {'song_id': 'tk0XbOHbmEQ', 'title': 'Pinocchio', 'duration': None, 'artists': ['Iniko'], 'album': ''}, {'song_id': 'Erp2J2g2d3A', 'title': 'i will be one of the greatest', 'duration': None, 'artists': ['TwentySixteen 意味'], 'album': ''}, {'song_id': 'SCW1-IMY60Y', 'title': 'Yosemite (Song For The Ahwahnechee)', 'duration': None, 'artists': ['Iniko'], 'album': ''}]


In [7]:
get_library = Tool(
    name = 'get_user_playlist',
    description= 'use this tool to get the full music playlist of the user. this gives all the playlsit existing library the user has. note the user is already authenticated and just pass a dummy user id if needed to the tool',
    func = yt_oath_user.get_user_library
)

In [8]:
result = get_library.invoke('demo')
print(result)

[
    {
        "title": "Liked Music",
        "playlistId": "LM",
        "description": "Auto playlist",
        "track_count": null
    },
    {
        "title": "Vimals",
        "playlistId": "PLZnYAMmCI5BY7zF-ShOUWie2qHgWuXQHo",
        "description": "AI Playground \u2022 374 tracks",
        "track_count": "374"
    },
    {
        "title": "Songs I grew Up with",
        "playlistId": "PLZnYAMmCI5BaA5DnAoll8wJILmJSec6Ya",
        "description": "AI Playground \u2022 197 tracks",
        "track_count": "197"
    },
    {
        "title": "Malayalam Songs for Travel",
        "playlistId": "PLZnYAMmCI5BbD3Vtv3LiuWLH_7Qw4hJR3",
        "description": "AI Playground \u2022 40 tracks",
        "track_count": "40"
    },
    {
        "title": "Malayalm Modern(Rap / Hip Hop)",
        "playlistId": "PLZnYAMmCI5BZUmXqXzPdffNe3PPzk_G7Y",
        "description": "AI Playground \u2022 28 tracks",
        "track_count": "28"
    },
    {
        "title": "malayalam",
        "playlistId

In [16]:
add_to_playlist = StructuredTool.from_function(
    name = 'add_to_playlist',
    description= 'use this tool to add a song to user playlist. you need to pass the playlist id and the song id or video id to the tool',
    func= yt_oath_user.add_song_to_playlist
)

In [ ]:
# result = add_to_playlist.invoke({
#     "playlist_id": "PLZnYAMmCI5BY7zF-ShOUWie2qHgWuXQHo",
#     "song_id": ["bVwPpsEqmm8"]
# })
# print(result)

{'status': 'STATUS_SUCCEEDED', 'playlistEditResults': [{'videoId': 'bVwPpsEqmm8', 'setVideoId': '2AAB0CD831B25FC9', 'multiSelectData': {'multiSelectParams': 'CAESIlBMWm5ZQU1tQ0k1Qlk3ekYtU2hPVVdpZTJxSGdXdVhRSG8=', 'multiSelectItem': 'Ch8KC2JWd1Bwc0VxbW04EhAyQUFCMENEODMxQjI1RkM5'}}]}


In [18]:
result = await get_library.ainvoke("vimal")
print(result)

[
    {
        "title": "Liked Music",
        "playlistId": "LM",
        "description": "Auto playlist",
        "track_count": null
    },
    {
        "title": "Vimals",
        "playlistId": "PLZnYAMmCI5BY7zF-ShOUWie2qHgWuXQHo",
        "description": "AI Playground \u2022 375 tracks",
        "track_count": "375"
    },
    {
        "title": "Songs I grew Up with",
        "playlistId": "PLZnYAMmCI5BaA5DnAoll8wJILmJSec6Ya",
        "description": "AI Playground \u2022 197 tracks",
        "track_count": "197"
    },
    {
        "title": "Malayalam Songs for Travel",
        "playlistId": "PLZnYAMmCI5BbD3Vtv3LiuWLH_7Qw4hJR3",
        "description": "AI Playground \u2022 40 tracks",
        "track_count": "40"
    },
    {
        "title": "Malayalm Modern(Rap / Hip Hop)",
        "playlistId": "PLZnYAMmCI5BZUmXqXzPdffNe3PPzk_G7Y",
        "description": "AI Playground \u2022 28 tracks",
        "track_count": "28"
    },
    {
        "title": "malayalam",
        "playlistId

In [20]:
llm_with_tools = llm.bind_tools([search_tool,get_library, add_to_playlist])

In [21]:
class State(TypedDict):
    messages : Annotated[List[Any], add_messages]

In [22]:
def music_search(state : State):
    response_from_llm = llm_with_tools.invoke(state['messages'])
    new_state = {
        'messages' : [response_from_llm]
    }
    return new_state

In [23]:
graph_builder = StateGraph(State)

In [24]:
graph_builder.add_node("musicbot" , music_search)
graph_builder.add_node("tools" , ToolNode([search_tool,get_library, add_to_playlist]))

In [25]:
graph_builder.add_edge(START, "musicbot")
graph_builder.add_conditional_edges("musicbot", tools_condition, "tools")
graph_builder.add_edge("tools", "musicbot")
graph_builder.add_edge("musicbot", END)

In [26]:
graph = graph_builder.compile(checkpointer=memory)

In [27]:
graph.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
         +----------+         
         | musicbot |         
         +----------+         
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


In [28]:
# sometimes this code will not work. this is an error with mermaid itself
display(graph.get_graph().draw_mermaid_png())

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xd8\x00\x00\x00\xf9\x08\x02\x00\x00\x00\xca\xb2l)\x00\x00\x10\x00IDATx\x9c\xec\x9d\x07|\x14E\xfb\xc7g\xf7Z\x92KH\xef!$!\x10\x08- \x88 \x02R\x04_:\xe2\x8b4A\xa4\xbf\x14\x05|A\xa5\x19\x90?\x02*/\x8a *""  \xbd\n"(\xc4 E0\x84\x1e\x12\x12B\x12H\xcf\xa5]\xd9\xfd?{\x9b\\.\xe1.$\xc8n\xe6n\xe7K>\xc7\xde\xcc\xec\xde\xdd\xecog\xe6y\xa6\xc9Y\x96E\x04B]#G\x04\x02\x06\x10!\x12\xb0\x80\x08\x91\x80\x05D\x88\x04, B$`\x01\x11"\x01\x0b\x88\x10\xab\xf20E\x7f\xe5\x8f\xec\xect\x9dN\xc7\xea\xb5\x06\x83\x16!\x1a!\x86\x8b\x92\xa9\x90\xa1\xd4\x98\xc8\x18B\xcbX\xc6@q\xe1\nd\xd0!\xf3\x03J\xc1\xb2:.\x8a\x96#F\xcf\x9f\xc2R\x88b\x8d\xd7\xa1\x15,c\x8c\xad\x94\xa0J\xb8\xd91\xf7V\x86\x18C\xc5\x97de,e\xa8\x88U8Pr%\xed\xe4,\xf3\x0fs|\xa6\xbb\x1b\xb2A(\xe2G\xe4I\xbdQzbgF~\xb6\x16\xf2C\xae\xa0\x95\x0e\xb4\xcaQF\xd1\xac\xbe\x94\xa1\x14\x885\xcaK\xae\xa2\xf4\xa5,\x02AQ )V\xae\xa4\xf4Z.\xf7dJ\xc4\xe9\x95;\xa0\x0c\xc6\x10ZE3\xa5\x9c\xe8dr\xca\xa07\xe6\xb0\x0c\xceAp\x16\x17\xab\

In [29]:
def process_ai_response(response):
    if "<think>" in response and "</think>" in response:
        # Extract everything cleanly
        before_think = response.split("<think>")[0]
        think_content = response.split("<think>")[1].split("</think>")[0]
        after_think = response.split("</think>")[1]

        # Format the think block for display
        formatted_think = f"> 💭 *{think_content.strip()}*"

        # Combine all parts
        return after_think.strip()
    else:
        return response

In [33]:
config =  {'configurable' : {'thread_id' : '3'}}

In [34]:
async def chat(message, history):
    current_state = {
        'messages' : message
    }

    result = await graph.ainvoke(current_state, config = config)

    return process_ai_response(result['messages'][-1].content)

In [35]:
gr.ChatInterface(chat, type = 'messages').launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


[
    {
        "title": "Liked Music",
        "playlistId": "LM",
        "description": "Auto playlist",
        "track_count": null
    },
    {
        "title": "Vimals",
        "playlistId": "PLZnYAMmCI5BY7zF-ShOUWie2qHgWuXQHo",
        "description": "AI Playground \u2022 376 tracks",
        "track_count": "376"
    },
    {
        "title": "Songs I grew Up with",
        "playlistId": "PLZnYAMmCI5BaA5DnAoll8wJILmJSec6Ya",
        "description": "AI Playground \u2022 197 tracks",
        "track_count": "197"
    },
    {
        "title": "Malayalam Songs for Travel",
        "playlistId": "PLZnYAMmCI5BbD3Vtv3LiuWLH_7Qw4hJR3",
        "description": "AI Playground \u2022 40 tracks",
        "track_count": "40"
    },
    {
        "title": "Malayalm Modern(Rap / Hip Hop)",
        "playlistId": "PLZnYAMmCI5BZUmXqXzPdffNe3PPzk_G7Y",
        "description": "AI Playground \u2022 28 tracks",
        "track_count": "28"
    },
    {
        "title": "malayalam",
        "playlistId